In [129]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [130]:
df = pd.read_csv("datasets\\sensor.csv")
df = df[df["machine_status"].isin(["NORMAL", "RECOVERING"])]
df["machine_status"] = df["machine_status"].replace({"NORMAL": 0, "RECOVERING": 1})
df.drop(columns=['sensor_15'], inplace=True)


C:\Users\Parvez\AppData\Local\Temp\ipykernel_19460\161507741.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["machine_status"] = df["machine_status"].replace({"NORMAL": 0, "RECOVERING": 1})


In [131]:
# If timestamp exists, sort by it (and then drop it)
if "timestamp" in df.columns:
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df = df.sort_values("timestamp")
    df = df.drop(columns=["timestamp"])

In [132]:
df.fillna(df.mean(), inplace=True)

In [133]:
df["machine_status"] = df["machine_status"].astype(int)

In [134]:
df["machine_status"].unique()

array([0, 1])

In [135]:
sensor_columns = [f"sensor_0{i}" for i in range(10)]
sensor_columns.extend([f"sensor_{i}" for i in range(10, 52)])
sensor_columns.remove("sensor_15")
label_column = "machine_status"   # Assumed binary label (after processing: e.g., 0 for normal, 1 for fault)

In [136]:
df[sensor_columns].max()

sensor_00       2.549016
sensor_01      56.727430
sensor_02      56.032990
sensor_03      48.220490
sensor_04     800.000000
sensor_05      99.999880
sensor_06      22.251160
sensor_07      23.596640
sensor_08      24.348960
sensor_09      25.000000
sensor_10      76.106860
sensor_11      60.000000
sensor_12      45.000000
sensor_13      31.187550
sensor_14     500.000000
sensor_16     739.741500
sensor_17     599.999939
sensor_18       4.873250
sensor_19     878.917900
sensor_20     448.907900
sensor_21    1107.526000
sensor_22     594.061100
sensor_23    1227.564000
sensor_24    1000.000000
sensor_25     839.575000
sensor_26    1214.420000
sensor_27    2000.000000
sensor_28    1841.146000
sensor_29    1466.281000
sensor_30    1600.000000
sensor_31    1800.000000
sensor_32    1839.211000
sensor_33    1578.600000
sensor_34     425.549800
sensor_35     694.479126
sensor_36     984.060700
sensor_37     174.901200
sensor_38     417.708300
sensor_39     547.916600
sensor_40     512.760400


In [137]:
df.fillna(df.mean(), inplace=True)

# Scale sensor columns
scaler = StandardScaler()
df[sensor_columns] = scaler.fit_transform(df[sensor_columns])

In [138]:
df[sensor_columns].max()

sensor_00     0.439173
sensor_01     2.773572
sensor_02     1.408796
sensor_03     1.847194
sensor_04     1.453500
sensor_05     1.538055
sensor_06     4.089005
sensor_07     3.567216
sensor_08     4.543642
sensor_09     4.928316
sensor_10     2.864207
sensor_11     1.384962
sensor_12     1.568508
sensor_13     3.493251
sensor_14     1.087796
sensor_16     2.564314
sensor_17     1.385069
sensor_18     3.355255
sensor_19     1.445221
sensor_20     0.864002
sensor_21     1.373361
sensor_22     0.868969
sensor_23     1.044995
sensor_24     2.434374
sensor_25     0.862269
sensor_26     1.735272
sensor_27     8.824201
sensor_28     3.160588
sensor_29     3.943207
sensor_30     5.037543
sensor_31     3.303555
sensor_32     3.971877
sensor_33     7.245212
sensor_34     2.156527
sensor_35     1.885840
sensor_36     1.351261
sensor_37     3.034647
sensor_38    34.999445
sensor_39    32.793238
sensor_40    20.786444
sensor_41    48.794452
sensor_42    33.035598
sensor_43    33.039799
sensor_44  

In [139]:
# Convert dataframe to numpy arrays
data_array = df[sensor_columns].values    # shape: (num_samples, num_features)
labels_array = df[label_column].values      # shape: (num_samples,)

In [141]:
class SlidingWindowDataset(Dataset):
    """
    Creates sliding windows from the sensor data.
    Each sample is a window of consecutive sensor readings and the label of the last time step.
    """
    def __init__(self, data, labels, window_size):
        self.data = data
        self.labels = labels
        self.window_size = window_size
        
    def __len__(self):
        return len(self.data) - self.window_size
    
    def __getitem__(self, idx):
        # Window of sensor readings
        X = self.data[idx : idx + self.window_size]
        # Label from the last time step in the window
        y = self.labels[idx + self.window_size - 1]
        return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

class CNNLSTM(nn.Module):
    def __init__(self, num_features, window_size):
        super(CNNLSTM, self).__init__()
        # CNN layers (Conv1d expects input shape: [batch, channels, sequence_length])
        self.conv1 = nn.Conv1d(in_channels=num_features, out_channels=64, kernel_size=5)
        self.relu = nn.ReLU()
        self.pool1 = nn.MaxPool1d(kernel_size=2)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding = 1)
        self.pool2 = nn.MaxPool1d(kernel_size=2)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3)
        self.pool3 = nn.MaxPool1d(kernel_size=2)

        self.lstm = nn.LSTM(input_size=256, hidden_size=128, num_layers=2, batch_first=True)
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, 10)
        self.fc3 = nn.Linear(10, 1)
        self.sigmoid = nn.Sigmoid()

        conv1_out = window_size - 5 + 1
        pool1_out = conv1_out // 2
        conv2_out = pool1_out
        pool2_out = conv2_out // 2
        conv3_out = pool2_out - 3 + 1
        pool3_out = conv3_out // 2

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.pool1(self.relu(self.conv1(x)))
        print("pool 1 x shape:", x.shape)
        x = self.pool2(self.relu(self.conv2(x)))
        print("pool 2 x shape:", x.shape)
        x = self.pool3(self.relu(self.conv3(x)))
        print("pool 3 x shape:", x.shape)

        x = x.permute(0, 2, 1)
        lstm_out, (h_n, _) = self.lstm(x)
        out = self.relu(self.fc1(h_n[-1]))
        out = self.relu(self.fc2(out))
        out = self.fc3(out)
        out = self.sigmoid(out)
        
        return out

### Hyperparameters

In [142]:
window_size = 100       # Number of consecutive readings per sample
batch_size = 32
num_epochs = 10
learning_rate = 0.001

In [143]:
dataset = SlidingWindowDataset(data_array, labels_array, window_size)
# Split the dataset into training and testing sets (90% train, 10% test)
train_size = int(0.9 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [145]:
len(train_loader), len(test_loader)

(6194, 689)

In [144]:
# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the CNN+LSTM model
num_features = len(sensor_columns)
model = CNNLSTM(num_features=num_features, window_size=window_size).to(device)

In [76]:
num_features    # Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [42]:
test = next(iter(train_loader))

In [44]:
test[0]

tensor([[[ 2.4607e-01,  5.8621e-01,  1.5406e-01,  ...,  2.3671e+00,
          -1.6201e-15,  2.5863e-01],
         [ 2.4607e-01,  5.8621e-01,  1.5406e-01,  ...,  2.2915e+00,
          -1.6201e-15,  2.3399e-01],
         [ 2.4363e-01,  5.8621e-01,  1.6590e-01,  ...,  2.2310e+00,
          -1.6201e-15,  2.1483e-01],
         ...,
         [ 1.9718e-01,  8.4975e-01,  1.5406e-01,  ...,  7.9501e-01,
          -1.6201e-15,  5.3310e-02],
         [ 2.0940e-01,  8.4975e-01,  1.5406e-01,  ...,  7.9501e-01,
          -1.6201e-15,  6.4261e-02],
         [ 1.9474e-01,  8.4975e-01,  1.4223e-01,  ...,  8.1013e-01,
          -1.6201e-15,  7.2474e-02]],

        [[ 8.4741e-02,  1.3636e+00,  7.9327e-01,  ...,  3.0473e+00,
          -1.6201e-15,  4.5574e-01],
         [ 6.7629e-02,  1.3636e+00,  8.0510e-01,  ...,  3.0171e+00,
          -1.6201e-15,  4.6395e-01],
         [ 7.9851e-02,  1.3636e+00,  7.9327e-01,  ...,  3.0473e+00,
          -1.6201e-15,  4.5300e-01],
         ...,
         [ 6.0297e-02,  1

In [84]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        # print("inputs:", inputs.shape)
        # print("labels:", labels.shape)
        optimizer.zero_grad()
        outputs = model(inputs)
        print("outputs:", outputs)
        loss = criterion(outputs, labels.unsqueeze(1))
        print(loss)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

pool 1 x shape: torch.Size([32, 64, 48])
pool 2 x shape: torch.Size([32, 128, 24])
pool 3 x shape: torch.Size([32, 256, 11])
outputs: tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], grad_fn=<SigmoidBackward0>)


RuntimeError: all elements of input should be between 0 and 1